In [171]:
import re
import os
import csv
import time
import datetime
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# 탱크옥션 자동 로그인
driver = webdriver.Chrome()
url = 'http://www.tankauction.co.kr/common/login_box.php'
driver.get(url)

In [172]:
driver.find_element_by_css_selector('#client_id').send_keys('kkj109')
## PW입력
driver.find_element_by_css_selector('#pwd_dummy').click()
time.sleep(2)
driver.find_element_by_name('passwd').send_keys('Kimriwon0710!')
## 로그인 버튼 클릭
driver.find_element_by_css_selector('#btn_login').click()
time.sleep(2)

In [173]:
## 종합검색 들어가기
driver.find_element_by_xpath("/html/body/div[1]/div[1]/div/div[2]/div/li[1]/a").click()
## 아파트선택
time.sleep(2)
driver.find_element_by_xpath("/html/body/div[2]/div[1]/div[3]/div[2]/div[1]/table/tbody/tr[2]/td[1]/select/option[3]").click()
## 매각기일(오늘부터 7일 후 선택)
time.sleep(2)
new_datetime = datetime.datetime.now() + datetime.timedelta(days=7)
new_datetime2 = new_datetime.strftime('%Y%m%d')
driver.find_element_by_xpath("/html/body/div[2]/div[1]/div[3]/div[2]/div[1]/table/tbody/tr[4]/td[2]/input[1]").send_keys(new_datetime2)
## 최저가격 조정
### 최저가, 3천만
time.sleep(2)
driver.find_element_by_xpath("/html/body/div[2]/div[1]/div[3]/div[2]/div[1]/table/tbody/tr[6]/td[1]/select[1]/option[14]").click()
### 최고가, 4억
time.sleep(2)
driver.find_element_by_xpath("/html/body/div[2]/div[1]/div[3]/div[2]/div[1]/table/tbody/tr[6]/td[1]/select[2]/option[14]").click()
## 검색 버튼 클릭
time.sleep(2)
driver.find_element_by_xpath("/html/body/div[2]/div[1]/div[3]/div[2]/div[1]/table/tbody/tr[8]/td/div[2]/input[15]").click()

In [181]:
driver.find_element_by_xpath('//*[@id="Prt_none"]/div/a[5]').click()

In [182]:
driver.find_element_by_xpath('//*[@id="list_tr_10"]').click()

In [183]:
#driver.find_element_by_xpath('//*[@id="list_tr_1"]').click()
driver.switch_to_window(driver.window_handles[1])
driver.get_window_position(driver.window_handles[1])

<ipython-input-183-c2a9649e0f0c>:2: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[1])


{'x': 0, 'y': 0}

In [184]:
tank_case_number_tmp = driver.find_element_by_xpath('//*[@id="view_cont"]/div[2]/div[2]/ul/li[2]').text
tank_case_number_tmp = tank_case_number_tmp.split()[1].split('(')
tank_case_number = tank_case_number_tmp[0]
try:
    tank_case_object = tank_case_number_tmp[1].replace(')',"")
except IndexError:
    tank_case_object = 1

print(tank_case_number_tmp)
print(tank_case_number)
print(tank_case_object)


['2019타경109681']
2019타경109681
1


In [185]:
tank_court = driver.find_element_by_xpath('//*[@id="view_cont"]/div[2]/div[2]/ul/li[3]').text
tank_court = tank_court.split()[0] + " " + tank_court.split()[1]
print(tank_court)

대전지방법원 본원


In [186]:
tank_death_day_tmp = driver.find_element_by_xpath('//*[@id="view_cont"]/div[5]/table/tbody/tr[1]/td/div[1]/div[2]').text
tank_death_day = tank_death_day_tmp.split()[1]
print(tank_death_day)

2020.11.16


In [187]:
tank_area_used_tmp = driver.find_element_by_xpath('//*[@id="view_cont"]/div[5]/table/tbody/tr[2]/td[1]/span').text
tank_area_used = tank_area_used_tmp.split()[0].replace('㎡','')
print(tank_area_used)

27.3


In [188]:
tank_site_tmp = driver.find_element_by_xpath('//*[@id="view_cont"]/div[5]/table/tbody/tr[3]/td[1]/span').text
tank_site = tank_site_tmp.split()[0].replace('㎡','')
print(tank_site)

16.89


In [189]:
tank_value_max = driver.find_element_by_xpath('//*[@id="view_cont"]/div[5]/table/tbody/tr[2]/td[3]').text
tank_value_max = tank_value_max.replace(',','')
print(tank_value_max)

42900000


In [190]:
tank_value_min = driver.find_element_by_xpath('//*[@id="view_cont"]/div[5]/table/tbody/tr[3]/td[3]/font').text
tank_value_min = tank_value_min.replace(',','')
print(tank_value_min)

30030000


In [197]:
tank_floor_top_tmp = driver.find_element_by_xpath('//*[@id="view_cont"]/div[8]/table/tbody/tr[2]/td[2]').text
print(tank_floor_top_tmp.split())
if len(tank_floor_top_tmp.split()) >= 3:
    tank_floor_top_tmp = tank_floor_top_tmp.split()[2]
    tank_floor_top = tank_floor_top_tmp.split('중')[0].replace('층', '').replace('(','')
    tank_floor_curr = tank_floor_top_tmp.split('중')[1].replace('층', '').replace(')','')
else:
    tank_floor_top = tank_floor_top_tmp.split('중')[0].replace('층', '')
    tank_floor_curr = tank_floor_top_tmp.split('중')[1].replace('층', '')
print(tank_floor_top)
print(tank_floor_curr)

['정림동', '642', '(13층중1층)']
(13층중1층)
정림동 642
13
1


In [152]:
tank_year_tmp = driver.find_element_by_xpath('//*[@id="view_cont"]/div[8]/table/tbody/tr[2]/td[3]').text
tank_year_tmp = tank_year_tmp.split('.')[0]
print(tank_year_tmp)
now_year = datetime.datetime.today().strftime('%y')
print(now_year)

if int(now_year) >= int(tank_year_tmp):
    tank_year = int(now_year) - int(tank_year_tmp) + 1
elif int(now_year) < int(tank_year_tmp):
    tank_year = int(now_year) - (int(tank_year_tmp) - 100) + 1
else:
    tank_year = "연도없음"
print(tank_year)

05
20
16


In [153]:
tank_addr_tmp = driver.find_element_by_xpath('//*[@id="view_cont"]/div[5]/table/tbody/tr[1]/td/div[2]').text
# tank_addr_tmp = tank_addr_tmp.split('전자지도')
tank_addr_tmp_00 = tank_addr_tmp.split('전자지도')[0].split(',')[0]
tank_addr_tmp_01 = tank_addr_tmp.split('전자지도')[0].split(',')[1].lstrip()
tank_addr_tmp_1 = tank_addr_tmp.split('전자지도')[1].split(':')[1].replace(')','').lstrip()

tank_addr_0 = tank_addr_tmp_00.split()[0]
tank_addr_1 = tank_addr_tmp_00.split()[1]
tank_addr_2 = tank_addr_tmp_00.split()[2]
tank_addr_3 = tank_addr_tmp_00.split()[3]
try:
    tank_addr_4 = tank_addr_tmp_00.split()[4]
except IndexError:
    tank_addr_4 = None
try:
    tank_addr_5 = tank_addr_tmp_00.split()[5]
except IndexError:
    tank_addr_5 = None
try:
    tank_addr_6 = tank_addr_tmp_00.split()[6]
except IndexError:
    tank_addr_6 = None
try:
    tank_addr_7 = tank_addr_tmp_00.split()[7]
except IndexError:
    tank_addr_7 = None
tank_apt = tank_addr_tmp_01.split()[0]
tank_apt_d = tank_addr_tmp_01.split()[1]
tank_apt_h = tank_addr_tmp_01.split()[len(tank_addr_tmp_01.split())-1]
tank_addr_d = tank_addr_tmp_1

#print(tank_addr_tmp)
print(tank_addr_tmp_00)
print(tank_addr_tmp_01)
print(tank_addr_tmp_1)

print("tank_addr_0 : ", format(tank_addr_0))
print("tank_addr_1 : ", format(tank_addr_1))
print("tank_addr_2 : ", format(tank_addr_2))
print("tank_addr_3 : ", format(tank_addr_3))
print("tank_addr_4 : ", format(tank_addr_4))
print("tank_addr_5 : ", format(tank_addr_5))
print("tank_addr_6 : ", format(tank_addr_6))
print("tank_addr_7 : ", format(tank_addr_7))
print("tank_apt : ", format(tank_apt))
print("tank_apt_d : ", format(tank_apt_d))
print("tank_apt_h : ", format(tank_apt_h))
print("tank_addr_d : ", format(tank_addr_d))

충청남도 계룡시 금암동 167-1
우림루미아트 107동 8층 802호 
충청남도 계룡시 장안로 75
tank_addr_0 :  충청남도
tank_addr_1 :  계룡시
tank_addr_2 :  금암동
tank_addr_3 :  167-1
tank_addr_4 :  None
tank_addr_5 :  None
tank_addr_6 :  None
tank_addr_7 :  None
tank_apt :  우림루미아트
tank_apt_d :  107동
tank_apt_h :  802호
tank_addr_d :  충청남도 계룡시 장안로 75


In [155]:
if tank_addr_0  == "서울특별시":
    tank_addr_0  = "서울시"
elif tank_addr_0  == "인천광역시":
    tank_addr_0 = "인천시"
elif tank_addr_0  == "부산광역시":
    tank_addr_0  = "부산시"
elif tank_addr_0  == "대전광역시":
    tank_addr_0  = "대전시"
elif tank_addr_0  == "대구광역시":
    tank_addr_0  = "대구시"
elif tank_addr_0  == "울산광역시":
    tank_addr_0  = "울산시"
elif tank_addr_0  == "세종특별자치시":
    tank_addr_0  = "세종시"
elif tank_addr_0  == "광주광역시":
    tank_addr_0  = "광주시"
elif tank_addr_0  == "제주특별자치도":
    tank_addr_0  = "제주시"

In [156]:
tank_list = [{
    '사건번호' : tank_case_number,
    '법원' : tank_court,
    '매각기일' : tank_death_day,
    '전용면적' : tanK_area_used,
    '대지권' : tank_site,
    '감정가' : tank_value_max,
    '최저가' : tank_value_min,
    '최고층' : tank_floor_top,
    '현재층' : tank_floor_curr,
    '사용승인(연차)' : tank_year,
    '주소0' : tank_addr_0,
    '주소1' : tank_addr_1,
    '주소2' : tank_addr_2,
    '주소3' : tank_addr_3,
    '주소4' : tank_addr_4,
    '주소5' : tank_addr_5,
    '주소6' : tank_addr_6,
    '주소7' : tank_addr_7,
    '아파트' : tank_apt,
    '동' : tank_apt_d,
    '호' : tank_apt_h,
    '도로명' : tank_addr_d,
    }]
df = pd.DataFrame(tank_list)
df = df[['사건번호', '법원', '매각기일', '전용면적', '대지권', '감정가', '최저가', '최고층', '현재층', '사용승인(연차)', '주소0', '주소1', '주소2', '주소3', '주소4', '주소5', '주소6', '주소7', '아파트', '동', '호', '도로명']]
df

,사건번호,법원,매각기일,전용면적,대지권,감정가,최저가,최고층,현재층,사용승인(연차),...,주소2,주소3,주소4,주소5,주소6,주소7,아파트,동,호,도로명
0,2018타경1939,대전지방법원 논산지원,2020.11.16,84.98㎡ (25.7평),54.15,195000000,195000000,15,8,16,...,금암동,167-1,None,None,None,None,우림루미아트,107동,802호,충청남도 계룡시 장안로 75
